In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from dateutil.rrule import rrule, SECONDLY, MINUTELY, HOURLY
from collections import defaultdict
from hrvanalysis import remove_outliers, remove_ectopic_beats, \
                        interpolate_nan_values, get_frequency_domain_features, \
                        get_time_domain_features, get_csi_cvi_features

from preprocessing.load_raw_vital_signs import *
from preprocessing.heart_rate_variability import calculate_hrv, apply_hrv

RAW_VITAL_DATA_PATH = "./DATA/Raw Data/filtered_df_removed_nan_files.parquet"
CLINICAL_DATA = './DATA/Clean Data/IMPALA_Clinical_Data_202308211019_Raw.csv'

### Select patients on hospital duration

In [ ]:

def read_clinical_df(path):
    """ Load clinical data into a Pandas DataFrame. """
    df = pd.read_csv(path, low_memory=False)
    smaller_df = df[['record_id', 'dis_outcome']]
    del df

    return smaller_df

df = read_clinical_df(CLINICAL_DATA)


In [ ]:

seed = 42
np.random.seed(seed)

alive_ids = []
died_ids = []

print('===== Select patients if they have at least 48 hours of data =====')

for patient, series in df.groupby('record_id'):
    if series.shape[0] >= 12:

        if series['dis_outcome'].iloc[0] == 1:
            alive_ids.append(patient)

        elif series['dis_outcome'].iloc[0] == 2:
            died_ids.append(patient)

print(f'- Number of alive patients that fit criteria: {len(alive_ids)}')
print(f'- Number of deceased patients that fit criteria: {len(died_ids)}\n')

alive_ids = np.random.choice(alive_ids, size=15, replace=False)
died_ids = np.random.choice(died_ids, size=15, replace=False)

print('===== Randomly select 15 patients from both selected groups ======')
print(f'Alive patients\n{alive_ids}\n')
print(f'Deceased patients\n{died_ids}')

del df

### Load data

In [ ]:

# Alive: B-N-0054, B-N-0091, Z-H-0009, Z-H-0014, Z-H-0211
# Died: B-N-0058, B-N-0009, B-N-0089, Z-H-0010, Z-H-0243


# data = read_raw_vital_signs(RAW_VITAL_DATA_PATH,
#                             batch_size=10000,
#                             patient_id=list(np.concatenate((alive_ids, died_ids))))


In [ ]:

# save_patient_dict(data, './DATA/Raw Data/raw_patient_dict_p30')

data = load_patient_dict('./DATA/Raw Data/raw_patient_dict_p30')


In [ ]:
print(data.keys())

### Window data

In [ ]:

def get_windowed_data(df, time_unit='m', time_freq=15):
    """
    Split the data into windows.
    :param df: Pandas DataFrame containing the data indexed on timestamps.
    :param time_unit: time unit of the data window, e.g. s (seconds), m (minutes).
    :param time_freq: number of time units in the data window.
    """

    df = df.copy()

    rrule_time = {'h' : HOURLY, 'm' : MINUTELY, 's' : SECONDLY}
    windowed_data = []
    
    for start in rrule(freq=rrule_time[time_unit], interval=time_freq, dtstart=df.index[0], until=df.index[-1]):
        
        end = start + pd.Timedelta(time_freq, unit=time_unit)
        idx = df.index.to_series().between(start, end)
        window = df[idx]
        windowed_data.append(window.values.squeeze())

    del df, window

    return windowed_data



In [ ]:

df = data['Z-H-0308'][['ECGHR', 'datetime']].copy()
df = df.set_index('datetime')
df = df.sort_index()

windowed_data = get_windowed_data(df, time_freq=15)

print(windowed_data[0].shape)


### Perform HRV analysis

In [ ]:

"""
Frequency-domain features: 'lf', 'hf', 'lf_hf_ratio', 'lfnu', 'hfnu',
                           'total_power', 'vlf'
Time-domain features: 'mean_nni', 'sdnn', 'sdsd', 'nni_50', 'pnni_50', 'nni_20',
                      'pnni_20', 'rmssd', 'median_nni', 'range_nni', 'cvsd',
                      'cvnni', 'mean_hr', 'max_hr', 'min_hr', 'std_hr'
Nonlinear-domain features: 'csi', 'cvi', 'Modified_csi'
"""

hrv_data = defaultdict(list)
threshold = 100
errors = 0

for patient_id in data:

    # Create windowed data
    df = data[patient_id][['ECGHR', 'datetime']].copy()
    df = df.set_index('datetime')
    df = df.sort_index()

    windowed_data = get_windowed_data(df, time_freq=15)

    for i, window in enumerate(windowed_data):

        if np.where(~np.isnan(window), 1, 0).sum() >= threshold:

            hrv_features = calculate_hrv(window)

            if hrv_features:
                print(hrv_features['lfnu'])
                hrv_data[patient_id].append(list(hrv_features.values()))
                break
                continue
        
        errors += 1

        break

    break


hrv_data = {k : np.array(v) for k, v in hrv_data.items()}



print(f'{errors} windows were too small or contained too many NaN values')


In [ ]:

from sklearn.preprocessing import normalize

# Remove rows that contain any NaN value
clean_hrv_data = {k : v[~np.isnan(v).any(axis=1)] for k, v in hrv_data.items()}

# Normalize data
normal_hrv_data = {k : normalize(v, axis=0) for k, v in clean_hrv_data.items()}

# All data into 1 array
all_hrv_data = np.concatenate([v for v in normal_hrv_data.values()])
print(all_hrv_data.shape)

normal_hrv_data_alive = np.concatenate([v[-192:, :] for k, v in normal_hrv_data.items() if \
                                       k in alive_ids])

normal_hrv_data_died = np.concatenate([v[-192:, :] for k, v in normal_hrv_data.items() if \
                                       k in died_ids])

normal_hrv_data_all = np.concatenate((normal_hrv_data_alive, normal_hrv_data_died))


In [ ]:

names = np.array(['lf', 'hf', 'lf_hf_ratio', 'lfnu', 'hfnu', 'total_power', 'vlf',
         'mean_nni', 'sdnn', 'sdsd', 'nni_50', 'pnni_50', 'nni_20', 'pnni_20',
         'rmssd', 'median_nni', 'range_nni', 'cvsd', 'cvnni', 'mean_hr',
         'max_hr', 'min_hr', 'std_hr', 'csi', 'cvi', 'Modified_csi'])


### Compare HRV metrics to paper

In [ ]:

clean_hrv_data = {k : v[~np.isnan(v).any(axis=1)] for k, v in hrv_data.items()}

clean_hrv_data_alive = np.concatenate([v[-192:, :] for k, v in clean_hrv_data.items() if \
                                       k in alive_ids])

clean_hrv_data_died = np.concatenate([v[-192:, :] for k, v in clean_hrv_data.items() if \
                                       k in died_ids])


In [ ]:

for i, feature in enumerate(clean_hrv_data_alive.T):
    
    print(names[i])
    print(f'IQR = {(np.quantile(feature, 0.75) - np.quantile(feature, 0.25))}')
    # print(np.quantile(feature, 0.5))
    # print(np.quantile(feature, 0.75))
    print()



### Perform PCA on HRV feature data

In [ ]:
from sklearn.decomposition import PCA

# Perform Principal Component Analysis
pca = PCA(n_components=5)
pca.fit(normal_hrv_data_all)
print(pca.get_feature_names_out())


In [ ]:

plt.imshow(pca.components_, cmap='magma', norm='linear')

idx = np.argsort(-pca.components_.sum(axis=0))
print(np.sort(pca.components_.sum(axis=0)))
print(np.array(names)[idx])

plt.colorbar()

plt.yticks(range(5), range(1, 6))
plt.ylabel('Nth component')
plt.xticks(range(len(names)), names, rotation=-90)
plt.title('Principal components and variable contribution')

plt.tight_layout()
plt.show()
plt.close()


### Feature selection: VarianceThreshold

In [ ]:
from sklearn.feature_selection import VarianceThreshold

def plot_variance(data, title):
    """
    Plot the variance of the given data.
    """

    selector = VarianceThreshold(threshold=0)
    selector.fit(data)

    plt.bar(range(7), selector.variances_[:7])
    plt.bar(range(7, 23), selector.variances_[7:23])
    plt.bar(range(23, 26), selector.variances_[23:26])
    plt.xticks(range(len(names)), names, rotation=-90)

    plt.ylabel('Variance')
    plt.xlabel('HRV features')
    plt.title(title)
    plt.legend(['Frequency-domain', 'Time-domain', 'Non Linear-domain'],
               bbox_to_anchor=(0.55, 0.75))

    plt.tight_layout()
    plt.show()
    plt.close()



In [ ]:

plot_variance(normal_hrv_data_alive,
              title=f'Variance per HRV feature; 15 patients (Alive), {len(normal_hrv_data_alive)} samples')

plot_variance(normal_hrv_data_died,
              title=f'Variance per HRV feature; 15 patients (Deceased), {len(normal_hrv_data_died)} samples')

plot_variance(normal_hrv_data_all,
              title=f'Variance per HRV feature; 30 patients (All), {len(normal_hrv_data_all)} samples')


In [ ]:

selector_alive = VarianceThreshold(threshold=0)
selector_alive.fit(normal_hrv_data_alive)

selector_died = VarianceThreshold(threshold=0)
selector_died.fit(normal_hrv_data_died)

selector_all = VarianceThreshold(threshold=0)
selector_all.fit(normal_hrv_data_all)

plt.bar(np.arange(-0.3, 25.7, 1), selector_alive.variances_, width=0.3, align='center')
plt.bar(range(26), selector_died.variances_, width=0.3, align='center')
plt.bar(np.arange(0.3, 26.3, 1), selector_all.variances_, width=0.3, align='center')
# plt.axhline(0.002, color='red')

plt.xticks(range(len(names)), names, rotation=-90)

plt.legend(['Alive', 'Deceased', 'All'], bbox_to_anchor=(0.9, 0.98))
plt.ylabel('Variance')
plt.xlabel('HRV features')
plt.show()
plt.close()


### Plot mean and standard deviation

In [ ]:
# 0, 1, 5, 6, 8, 10, 11, 12, 13, 26

a_mean = np.array([np.nanmean(v[-192:, :], axis=0) for k, v in normal_hrv_data.items() if \
     k in alive_ids])

b_mean = np.array([np.nanmean(v[-192:, :], axis=0) for k, v in normal_hrv_data.items() if \
    k in died_ids])

x = np.arange(26)

a = plt.boxplot(a_mean,
             positions=x-0.15,
             widths=0.25,
             showfliers=False,
             patch_artist=True,
             boxprops=dict(facecolor='tab:green', color='black'),
             medianprops=dict(color='black'),
             showmeans=True,
             meanprops=dict(markerfacecolor='black', markeredgecolor='black'))

b = plt.boxplot(b_mean,
             positions=x+0.15,
             widths=0.23,
             showfliers=False,
             patch_artist=True,
             boxprops=dict(facecolor='tab:red', color='black'),
             medianprops=dict(color='black'),
             showmeans=True,
             meanprops=dict(markerfacecolor='black', markeredgecolor='black'))

plt.xticks(range(26), names, rotation=-90)

plt.legend([a['boxes'][0], b['boxes'][0], a['means'][0]], ['Alive', 'Deceased', 'Mean'])
plt.title('Average of 30 patients (15 alive, 15 deceased)')
plt.tight_layout()
plt.show()


In [ ]:
x = np.arange(26)

# [:, [0, 1, 5, 6, 8, 10, 11, 12, 13, 16, 18, 25]]

a_std = np.array([np.nanstd(v[-192:, :], axis=0) for k, v in normal_hrv_data.items() if \
     k in alive_ids])

b_std = np.array([np.nanstd(v[-192:, :], axis=0) for k, v in normal_hrv_data.items() if \
    k in died_ids])

a = plt.boxplot(a_std,
             positions=x-0.15,
             widths=0.25,
             showfliers=False,
             patch_artist=True,
             boxprops=dict(facecolor='tab:green', color='black'),
             medianprops=dict(color='black'),
             showmeans=True,
             meanprops=dict(markerfacecolor='black', markeredgecolor='black'))

b = plt.boxplot(b_std,
             positions=x+0.15,
             widths=0.23,
             showfliers=False,
             patch_artist=True,
             boxprops=dict(facecolor='tab:red', color='black'),
             medianprops=dict(color='black'),
             showmeans=True,
             meanprops=dict(markerfacecolor='black', markeredgecolor='black'))

# plt.xticks(x, ['lf', 'hf', 'total\n_power', 'vlf', 'sdnn', 'nni\n_50', 'pnni\n_50',
#                'nni\n_20', 'pnni\n_20', 'range\n_nni', 'cvnni', 'modi\nfied\n_csi'])

plt.xticks(range(26), names, rotation=-90)

plt.legend([a['boxes'][0], b['boxes'][0], a['means'][0]], ['Alive', 'Deceased', 'Mean'])
plt.title('Standard deviation of 30 patients (15 alive, 15 deceased)')
plt.tight_layout()
plt.show()

### Plot zero counts per feature

In [ ]:

plt.bar(np.arange(-0.4, 25.6, 1),
        np.count_nonzero(normal_hrv_data_alive == 0, axis=0),
        width=0.4, align='edge', label='Alive (count)')

plt.bar(range(26),
        np.count_nonzero(normal_hrv_data_died == 0, axis=0),
        width=0.4, align='edge', label='Deceased (count)')

plt.axhline(len(normal_hrv_data_died), color='tab:red', label='Max')

plt.xticks(range(26), names, rotation=-90)

plt.legend()
plt.ylabel('Variance')
plt.xlabel('HRV features')
plt.title('Zero count per feature')
plt.tight_layout()
plt.show()
plt.close()


### Plot histograms of HRV features (alive vs. died)

In [ ]:
import seaborn as sns

fig = plt.figure(figsize=(10, 10))# 0, 3, 5, 6, 12, 17, 19, 22

j = 0
for i, feature in enumerate(names):

    if i in [3, 19, 12, 22, 17, 21, 20, 5]:#[19, 12,  3, 22, 17]: #[3, 5, 12, 17, 19, 22]:#[1, 2, 3, 5, 6, 12, 16, 17, 19, 22]:

        plt.subplot(3, 3, j+1)
        j += 1

        # sns.histplot(normal_hrv_data_alive[:, i],
        #              kde=True, bins=50, color='tab:blue')
        # sns.histplot(normal_hrv_data_died[:, i],
        #              kde=True, bins=50, color='tab:orange')

        sns.histplot(normal_hrv_data_alive[:, i][normal_hrv_data_alive[:, i] <= 0.2],
                     kde=True, bins=50, color='tab:blue')
        sns.histplot(normal_hrv_data_died[:, i][normal_hrv_data_died[:, i] <= 0.2],
                     kde=True, bins=50, color='tab:orange')
        plt.xlim(-0.01, 0.21)

        plt.title(feature)
        plt.legend(['Alive', 'Deceased'])


plt.tight_layout()
plt.show()


In [ ]:

a = sns.histplot(normal_hrv_data_alive[:, 4][normal_hrv_data_alive[:, 4] <= 0.2], kde=True, bins=100,
                    color='tab:blue', element="step", label='Alive')
b = sns.histplot(normal_hrv_data_died[:, 4][normal_hrv_data_died[:, 4] <= 0.2], kde=True, bins=100,
                color='tab:orange', element="step", label='Deceased')

plt.title(f'hf (zoomed in)')
# plt.xlim(-0.01, 0.21)
plt.legend()
plt.tight_layout()
plt.show()


In [ ]:

for i, feature in enumerate(names):     #[2, 3, 4, 7, 8, 15, 17, 19, 21, 23, 24]

    # if i in [0, 3, 5, 6, 12, 17, 19, 22]: #[2, 9, 14, 16, 18]:
    
    fig = plt.figure(figsize=(10, 4))

    plt.subplot(1, 2, 1)
    a = sns.histplot(normal_hrv_data_alive[:, i], kde=True, bins=100,
                    color='tab:blue', element="step", label='Alive')
    b = sns.histplot(normal_hrv_data_died[:, i], kde=True, bins=100,
                    color='tab:orange', element="step", label='Deceased')

    plt.title(f'{feature}')
    plt.legend()
    plt.tight_layout()


    plt.subplot(1, 2, 2)
    a = sns.histplot(normal_hrv_data_alive[:, i][normal_hrv_data_alive[:, i] <= 0.2], kde=True, bins=100,
                    color='tab:blue', element="step", label='Alive')
    b = sns.histplot(normal_hrv_data_died[:, i][normal_hrv_data_died[:, i] <= 0.2], kde=True, bins=100,
                    color='tab:orange', element="step", label='Deceased')

    plt.title(f'{feature} (zoomed in)')
    plt.xlim(-0.01, 0.21)
    plt.legend()
    plt.tight_layout()
    plt.show()


### KL divergence

In [ ]:
from sklearn.metrics import mutual_info_score
from scipy.special import rel_entr

sum_to_1_alive = (normal_hrv_data_alive / normal_hrv_data_alive.sum(axis=0))
sum_to_1_died = (normal_hrv_data_died / normal_hrv_data_died.sum(axis=0))

info_s = []
info_e = []

for i in [2, 3, 7, 9, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]:#[2, 9, 14, 16, 18]:

    # Mututal information score (higher the more alike, 0 if completely independent)
    a = np.histogram(sum_to_1_alive[:, i], bins=100)[0]
    b = np.histogram(sum_to_1_died[:, i], bins=100)[0]
    s = mutual_info_score(a, b)
    info_s.append(s)

    # Entropy (lower is more alike, 0 if identical)
    e = sum(rel_entr(sum_to_1_alive[:, i], sum_to_1_died[:2490, i]))
    info_e.append(e)

info_s = np.array(info_s)
info_e = np.array(info_e)


plt.scatter(range(15), info_s, color='tab:blue')
plt.scatter(range(15), info_e, color='tab:orange')
# plt.plot(range(5), np.abs(info_s - info_e), color='tab:green')
plt.xticks(range(15), np.array(names)[[2, 3, 7, 9, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]], rotation=-90)
plt.legend(['Mutual information (low)', 'KL-divergence (high)', 'Difference'])
plt.title('Distribution similarity between alive and deceased patients')
plt.tight_layout()
plt.show()
plt.close()


In [ ]:
info_s = []
info_e = []

for i in range(26):

    # Mututal information score (higher the more alike, 0 if completely independent)
    a = np.histogram(sum_to_1_alive[:, i], bins=100)[0]
    b = np.histogram(sum_to_1_died[:, i], bins=100)[0]
    s = mutual_info_score(a, b)
    info_s.append(s)

    # Entropy (lower is more alike, 0 if identical)
    e = sum(rel_entr(sum_to_1_alive[:, i], sum_to_1_died[:2490, i]))
    info_e.append(e)

info_s = np.array(info_s)
info_e = np.array(info_e)


plt.scatter(range(26), info_s, color='tab:blue')
plt.scatter(range(26), info_e, color='tab:orange')
plt.xticks(range(26), names, rotation=-90)
plt.legend(['Mutual information (low)', 'KL-divergence (high)', 'Difference'])
plt.title('Distribution similarity between alive and deceased patients')
plt.tight_layout()
plt.show()
plt.close()


### Feature selection: Sklearn

In [ ]:
from sklearn.feature_selection import SelectFromModel
from sklearn.svm import LinearSVC
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.linear_model import LogisticRegression, RidgeCV, LassoCV, SGDClassifier
from sklearn.feature_selection import RFE, RFECV
from sklearn.svm import SVC
from sklearn.feature_selection import VarianceThreshold

y_alive = np.ones(normal_hrv_data_alive.shape[0])
y_died = np.zeros(normal_hrv_data_died.shape[0])

X = np.concatenate((normal_hrv_data_alive, normal_hrv_data_died))
y = np.concatenate((y_alive, y_died))

print(normal_hrv_data_alive.shape)
print(normal_hrv_data_died.shape)
print(y_alive.shape)
print(y_died.shape)


In [ ]:

clfs = [LogisticRegression(),
        LinearSVC(dual='auto'),
        ExtraTreesClassifier(n_estimators=50)]
        # SGDClassifier(max_iter=10000)]

counter = np.zeros(len(names))

for clf in clfs:

    print(clf)

    fitted_clf = clf.fit(X, y)
    model_clf = SelectFromModel(fitted_clf, prefit=True, max_features=10)
    print(names[model_clf.get_support()])
    counter += model_clf.get_support().astype(int)

    rfe_clf = RFE(estimator=clf, n_features_to_select=10)
    rfe_clf.fit(X, y)
    print(names[rfe_clf.get_support()])
    counter += rfe_clf.get_support().astype(int)

    print()


plt.bar(range(len(names)), counter)
plt.xticks(range(len(names)), names, rotation=-90)
plt.show()

np.argsort(-counter)


In [ ]:

# Classifiers
lsvc_l1 = LinearSVC(dual="auto", penalty="l1", max_iter=10000)
lsvc_l2 = LinearSVC(dual="auto", penalty="l2", max_iter=10000)
sgd = SGDClassifier(penalty='l1', max_iter=10000)
svc = SVC(kernel='linear', max_iter=-1)
random_forest = ExtraTreesClassifier(n_estimators=50)

ridge = RidgeCV()
lasso = LassoCV(max_iter=50000)


In [ ]:

lsvc_l1 = lsvc_l1.fit(X, y)
model_lsvc_l1 = SelectFromModel(lsvc_l1, prefit=True, max_features=5)
print(names[model_lsvc_l1.get_support()])

lsvc_l2 = lsvc_l2.fit(X, y)
model_lsvc_l2 = SelectFromModel(lsvc_l2, prefit=True, max_features=5)
print(names[model_lsvc_l1.get_support()])

sgd = sgd.fit(X, y)
model_sgd = SelectFromModel(sgd, prefit=True, max_features=5)
print(names[model_sgd.get_support()])

svc = svc.fit(X, y)
model_svc = SelectFromModel(svc, prefit=True, max_features=5)
print(names[model_svc.get_support()])

random_forest = random_forest.fit(X, y)
model_tree = SelectFromModel(random_forest, prefit=True, max_features=5)
print(names[model_tree.get_support()])



In [ ]:

ridge = ridge.fit(X, y)

sfm_ridge = SelectFromModel(ridge, max_features=5)
print(names[sfm_ridge.get_support()])

sfs_forward_ridge = SequentialFeatureSelector(ridge, n_features_to_select=5, direction='forward').fit(X, y)
print(names[sfs_forward_ridge.get_support()])

sfs_backward_ridge = SequentialFeatureSelector(ridge, n_features_to_select=5, direction='backward').fit(X, y)
print(names[sfs_backward_ridge.get_support()])


lasso = lasso.fit(X, y)

sfm_lasso = SelectFromModel(lasso, max_features=5)
print(names[sfm_lasso.get_support()])

sfs_forward_lasso = SequentialFeatureSelector(lasso, n_features_to_select=5, direction='forward').fit(X, y)
print(names[sfs_forward_lasso.get_support()])

sfs_backward_lasso = SequentialFeatureSelector(lasso, n_features_to_select=5, direction='backward').fit(X, y)
print(names[sfs_backward_lasso.get_support()])


In [ ]:

svc = SVC(kernel="linear")
rfe = RFE(estimator=svc, n_features_to_select=5, step=1)
rfe.fit(X, y)

print(names[rfe.get_support()])


In [ ]:

sel = VarianceThreshold(threshold=0.004).fit(X, y)
print(names[sel.get_support()])


In [ ]:

counter = np.zeros(len(names))
counter += model_lsvc_l1.get_support().astype(int)
counter += model_lsvc_l2.get_support().astype(int)
counter += model_sgd.get_support().astype(int)
counter += model_svc.get_support().astype(int)
counter += model_tree.get_support().astype(int)
counter += sfm_ridge.get_support().astype(int)
counter += sfs_forward_ridge.get_support().astype(int)
counter += sfs_backward_ridge.get_support().astype(int)
counter += sfm_lasso.get_support().astype(int)
counter += sfs_forward_lasso.get_support().astype(int)
counter += sfs_backward_lasso.get_support().astype(int)
counter += rfe.get_support().astype(int)
counter += sel.get_support().astype(int)

print(counter)
plt.bar(range(len(names)), counter)
plt.xticks(range(len(names)), names, rotation=-90)
plt.show()

# hf, lf_hf_ratio, lfnu, nni_20, pnni_20, cvsd, mean_hr, std_hr
# 1, 2, 3, 11, 12, 13, 17, 19, 22
# 0, 3, 5, 6, 12, 17, 19, 22
np.argsort(-counter)


In [ ]:
# print(names[[2, 3, 4, 7, 8, 15, 17, 19, 21, 23, 24]])
# print()
# print(names[[1, 2, 3, 11, 12, 13, 17, 19, 22]])

print([n for n in names[[1, 2, 3, 11, 12, 13, 17, 19, 22]] if \
       n in names[[2, 3, 4, 7, 8, 15, 17, 19, 21, 23, 24]]])